In [2]:
# In this example we will show the difference between fixes computed with laika
# from raw data of the ublox receiver vs the the fixes the ublox receiver
# computes

In [7]:
import numpy as np
from pathlib import Path

base_dir = Path('example_data')
raw_ublox_t = np.load(base_dir / 'raw_gnss_ublox/t')
raw_ublox = np.load(base_dir / 'raw_gnss_ublox/value')
fixes_ublox_t = np.load(base_dir / 'live_gnss_ublox/t')
fixes_ublox = np.load(base_dir / 'live_gnss_ublox/value')
print(raw_ublox_t)

[46408.67372125 46408.67372125 46408.67372125 ... 46468.57372125
 46468.57372125 46468.57372125]


In [2]:
# We get the raw data into our format from the log array format
import sys
sys.path.append('../')
from laika.raw_gnss import normal_meas_from_array

measurements = np.array([normal_meas_from_array(arr) for arr in raw_ublox])

In [3]:
# initialize an astrodog with dgps corrections

from laika import AstroDog

dog = AstroDog(dgps=True)

In [6]:
# Building this cache takes forever, just copy it from repo

from shutil import copyfile

cache_directory = Path(dog.cache_dir) / 'cors_coord'
cache_directory.mkdir(parents=True, exist_ok=True)
copyfile('cors_station_positions', cache_directory / 'cors_station_positions')

PosixPath('/tmp/gnss/cors_coord/cors_station_positions')

In [6]:
from laika.raw_gnss import process_measurements, correct_measurements, calc_pos_fix
from tqdm.auto import tqdm

# We want to group measurements by measurement epoch
# this makes the kalman filter faster and is easier
# to reason about
grouped_t = np.unique(raw_ublox_t)
grouped_meas_processed = []
corrected_meas_arrays = []

# process measurement groups
for t in grouped_t:
    meas = measurements[raw_ublox_t == t]
    # print("1",type(meas[-1].sat_ephemeris), meas[-1].sat_ephemeris, meas[-1].sat_ephemeris.prn)
    suanz_meas = meas[0]
    print(suanz_meas.recv_time_week, suanz_meas.recv_time_sec)
    grouped_meas_processed.append(process_measurements(meas, dog))
    suanz_meas = grouped_meas_processed[-1][0]
    print(suanz_meas.recv_time_week, suanz_meas.recv_time_sec)
    # print("2", suanz_meas.sat_ephemeris.eph_type, suanz_meas.sat_ephemeris.prn, suanz_meas.sat_ephemeris.data, suanz_meas.sat_ephemeris.get_sat_info())

# correct measurement groups with an estimate position
# that was computes with weighted-least-squares on
# the first epoch
# WARNING: can take up to 10min

2012.0 404106.397
2012.0 404106.397
2012.0 404106.49700000003
2012.0 404106.49700000003
2012.0 404106.597
2012.0 404106.597
2012.0 404106.697
2012.0 404106.697
2012.0 404106.797
2012.0 404106.797
2012.0 404106.897
2012.0 404106.897
2012.0 404106.99700000003
2012.0 404106.99700000003
2012.0 404107.097
2012.0 404107.097
2012.0 404107.197
2012.0 404107.197
2012.0 404107.297
2012.0 404107.297
2012.0 404107.397
2012.0 404107.397
2012.0 404107.49700000003
2012.0 404107.49700000003
2012.0 404107.597
2012.0 404107.597
2012.0 404107.697
2012.0 404107.697
2012.0 404107.797
2012.0 404107.797
2012.0 404107.897
2012.0 404107.897
2012.0 404107.99700000003
2012.0 404107.99700000003
2012.0 404108.097
2012.0 404108.097
2012.0 404108.197
2012.0 404108.197
2012.0 404108.297
2012.0 404108.297
2012.0 404108.397
2012.0 404108.397
2012.0 404108.49700000003
2012.0 404108.49700000003
2012.0 404108.597
2012.0 404108.597
2012.0 404108.697
2012.0 404108.697
2012.0 404108.797
2012.0 404108.797
2012.0 404108.897
20

  0%|          | 0/600 [00:00<?, ?it/s]

HTTPS error 404
HTTPS error 404
download_cors_station, File not downloaded, check availability on server.
HTTPS error 404
HTTPS error 404
download_cors_station, File not downloaded, check availability on server.
HTTPS error 404
HTTPS error 404
download_cors_station, File not downloaded, check availability on server.


In [ ]:
wls_estimate = calc_pos_fix(grouped_meas_processed[0])
est_pos = wls_estimate[0][:3]
for proc in tqdm(grouped_meas_processed):
    corrected = correct_measurements(proc, est_pos, dog)
    corrected_meas_arrays.append(np.array([c.as_array() for c in corrected]))

In [8]:
for proc in tqdm(grouped_meas_processed):
    corrected = correct_measurements(proc, est_pos, dog)
    corrected_meas_arrays.append(np.array([c.as_array() for c in corrected]))

  0%|          | 0/600 [00:00<?, ?it/s]

In [9]:
# Generate and build a CFFI library from the Kalman filter defined symbolically in SymPy
from kalman.models.gnss_kf import GNSSKalman

GNSSKalman.generate_code()
!g++ kalman/generated/gnss.cpp -Wall -fPIC -shared -o kalman/generated/libgnss.so

In [10]:
# We run the Kalman filter

from kalman.kalman_helpers import run_car_ekf_offline, ObservationKind

ekf = GNSSKalman()
init_state = ekf.x
init_state[:3] = est_pos
ekf.init_state(init_state)
ekf_data = {}
ekf_data[ObservationKind.PSEUDORANGE_GPS] = (grouped_t, corrected_meas_arrays)
ekf_data[ObservationKind.PSEUDORANGE_RATE_GPS] = (grouped_t, corrected_meas_arrays)
ekf_outputs = run_car_ekf_offline(ekf, ekf_data)

import laika.lib.coordinates as coord

laika_positions_t = ekf_outputs[4]
laika_positions_ecef = ekf_outputs[0][:, :3]
laika_positions_geodetic = coord.ecef2geodetic(laika_positions_ecef)

  0%|          | 0/1200 [00:00<?, ?it/s]

In [11]:
ublox_positions_geodetic = fixes_ublox[:, [0, 1, 4]]

In [12]:
# By looking at the map, we can see that the two paths compared.
# If you want to regenerate the gmplot you will need a google
# maps API key

import gmplot

gmap = gmplot.GoogleMapPlotter(*laika_positions_geodetic[0])
#gmap.apikey='...'
gmap.plot([x[0] for x in laika_positions_geodetic], [x[1] for x in laika_positions_geodetic], 'blue', edge_width=5)
gmap.plot([x[0] for x in ublox_positions_geodetic], [x[1] for x in ublox_positions_geodetic], 'red', edge_width=5)
gmap.draw("laika_quality_check.html")

import webbrowser
import os

webbrowser.open('file://' + os.path.realpath("laika_quality_check.html"));